In [3]:
from ultralytics import YOLO
import cv2
import numpy as np
import time
import threading
import socket
import queue
import sys


In [19]:

position_data = queue.LifoQueue()

cap = cv2.VideoCapture(0)
face_model = YOLO("yolov11s-face.pt")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
x_axis = int(width/2)
y_axis = int(height/2)
def image_processing(): #image processing function   
    while True:
        print("before entering yolo")
        ret , frame = cap.read()
        results = face_model(frame)[0]
        for result in results.boxes.data.tolist():
                               
             x1, y1, x2, y2, score, class_id = result #for every result
             
             if score > 0.3: #if confidence score > 0.3
                 
                #to find the center by calculating the distance between 2 points
                distanceX = x2-x1 
                distanceY = y2-y1
                centerX = distanceX/2
                centerY = distanceY/2

                #adding the center to full image
                actualX = centerX + x1
                actualY = centerY + y1
                position_data.put((x1, y1, x2, y2, score, class_id, actualX , actualY , centerX , centerY))
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 3) #face detection
                cv2.circle(frame , (int(actualX),  int(actualY)) , 7 , (0 , 0 , 255) , -1) #center
                cv2.putText(frame, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)), #write FACE
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)
                cv2.putText(frame, str(round(float(score) , 3)), (int(x1), int(y2+20)), #confidence score
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 3, cv2.LINE_AA)
              
                
        cv2.line(frame, (x_axis , 0) , (x_axis , height) , (255 , 0 , 0) , 2) #line to cut the image vertically
        cv2.line(frame , (0 , y_axis) , (width , y_axis) , (255 , 0 , 0) , 2) #line to cut the image horizontally
        cv2.imshow("frame" , frame)
        if cv2.waitKey(15) == 27: #esc key
             break

    
    cap.release()
    cv2.destroyAllWindows()

In [12]:
def connect_to_esp32(HOST , PORT): #to establish connection.
    counter = 0
    while True:
        try:
            print("Connecting to esp32....")
            sock = socket.socket(socket.AF_INET , socket.SOCK_DGRAM)
            sock.connect((HOST , PORT))
            print("Connection is established!! :)")
            return sock
        except socket.error as e:
            
            print(f"Connection faild, the error is {e}.... Retrying in 4 seconds")
            time.sleep(4)
            counter = counter + 1
            if counter > 4: #will try 4 times
                return None 
            
        

In [17]:
def send_to_esp(HOST, PORT ,  sock , x1, y1, x2, y2, score, class_id, actualX , actualY , centerX , centerY):
    try:
       message = f"{x1},{y1},{x2},{y2},{score},{class_id},{actualX},{actualY},{centerX},{centerY}\n"
       sock.sendto(message.encode("utf-8"), (HOST, PORT))
       print("Data sent to esp:", message)
    except (socket.error , BrokenPipeError) as e:
       print("Error:" , e)
       raise

In [15]:

def main():
    position_data.queue.clear()
    print("Starting Main.....")
    HOST = "192.168.8.187"
    PORT = 20056
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    if sock is None:
        sys.exit("Unable to connect") #shut down
    thread1 = threading.Thread(target=image_processing) #to start the function and do while loop in the same time
    thread1.start()
    print("started thread")
    while thread1.is_alive():
        try:
            
            if not position_data.empty():
               x1, y1, x2, y2, score, class_id, actualX , actualY , centerX , centerY = position_data.get()
                
               send_to_esp(HOST , PORT ,sock , x1, y1, x2, y2, score, class_id, actualX , actualY , centerX , centerY)
               time.sleep(0.25) 
        except Exception as e: #if connection error occurs
            print(f"Something went wrong {e} , Reconnecting again...")
            sock.close()
            sock = connect_to_esp32(HOST , PORT)
            while not position_data.empty():
                position_data.get() #to clear it
                print("position data is cleared")
            if sock is None:
                print("Can not connect. Exiting.... :(")
                break      
         
                
        # start = len(position_data)
        # if start > 0:
        #     del position_data[0:start-1]
        #     start = len(position_data)
        #     print(position_data[start-1])
        #     x1, y1, x2, y2, score, class_id, actualX , actualY , centerX , centerY = position_data[start-1] #tuple unpacking
        #     send_to_esp(sock , x1, y1, x2, y2, score, class_id, actualX , actualY , centerX , centerY)
            
    
        
    
    thread1.join()
    print("finished")

In [20]:
main()

Starting Main.....
before entering yolo
started thread

0: 480x640 1 face, 6081.7ms
Speed: 109.6ms preprocess, 6081.7ms inference, 543.2ms postprocess per image at shape (1, 3, 480, 640)
Data sent to esp: 310.3320007324219,161.56939697265625,406.1881408691406,305.99249267578125,0.8546565771102905,0.0,358.26007080078125,233.78094482421875,47.928070068359375,72.2115478515625

before entering yolo

0: 480x640 1 face, 128.2ms
Speed: 1.9ms preprocess, 128.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
before entering yolo

Data sent to esp: 427.4144287109375,219.61264038085938,513.3335571289062,318.0423278808594,0.7832969427108765,0.0,470.3739929199219,268.8274841308594,42.959564208984375,49.21484375

0: 480x640 1 face, 120.2ms
Speed: 1.9ms preprocess, 120.2ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)
before entering yolo

0: 480x640 1 face, 114.3ms
Speed: 1.6ms preprocess, 114.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
